<a href="https://colab.research.google.com/github/apangee/UCONN-MATH5671/blob/master/Assignment9_Group4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Drive mounting

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


Something is wrong with Spark 2.4.4 and Java 8. Should use lower version 2.2.3. Also only Java 8 works. 

In [0]:
!wget -q https://archive.apache.org/dist/spark/spark-2.2.3/spark-2.2.3-bin-hadoop2.7.tgz
!tar xf spark-2.2.3-bin-hadoop2.7.tgz
!pip install -q findspark

In [3]:
!ls

gdrive	sample_data  spark-2.2.3-bin-hadoop2.7	spark-2.2.3-bin-hadoop2.7.tgz


Install Java 8 if Java 11 was installed. Spark is compatible with Java 8 now

In [4]:
import os       #importing os to set environment variable
def install_java():
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null      #install openjdk
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     #set environment variable
  !java -version       #check java version
install_java()

openjdk version "11.0.4" 2019-07-16
OpenJDK Runtime Environment (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3)
OpenJDK 64-Bit Server VM (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3, mixed mode, sharing)


Switch to Java 8 if it is Java 11 now

In [5]:
!update-alternatives --config java

There are 2 choices for the alternative java (providing /usr/bin/java).

  Selection    Path                                            Priority   Status
------------------------------------------------------------
* 0            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      auto mode
  1            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      manual mode
  2            /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java   1081      manual mode

Press <enter> to keep the current choice[*], or type selection number: 2
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode


Double check if it is Java 8 now

In [6]:
!java -version 

openjdk version "1.8.0_222"
OpenJDK Runtime Environment (build 1.8.0_222-8u222-b10-1ubuntu1~18.04.1-b10)
OpenJDK 64-Bit Server VM (build 25.222-b10, mixed mode)


Set some environment variables

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.2.3-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()

In [0]:
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
conf = SparkConf().setAppName("MLlib")
sc = SparkContext(conf=conf)
sc.setLogLevel("WARN")
spark = SparkSession(sc)

Reading Data

In [10]:
payment_df = spark.read.format("csv").option("header", "true").load(r'/content/gdrive/My Drive/Uconn Each Semester/2019 Fall/Math 5671/Week9/data/payment.csv')
payment_df = payment_df.alias('payment_df')
payment_df.show(3)

+----------+-----------+--------+---------+------+-------------------+-------------------+
|payment_id|customer_id|staff_id|rental_id|amount|       payment_date|        last_update|
+----------+-----------+--------+---------+------+-------------------+-------------------+
|         1|          1|       1|       76|  2.99|2005-05-25 11:30:37|2006-02-15 22:12:30|
|         2|          1|       1|      573|  0.99|2005-05-28 10:35:23|2006-02-15 22:12:30|
|         3|          1|       1|     1185|  5.99|2005-06-15 00:54:12|2006-02-15 22:12:30|
+----------+-----------+--------+---------+------+-------------------+-------------------+
only showing top 3 rows



In [11]:
rental_df = spark.read.format("csv").option("header", "true").load(r'/content/gdrive/My Drive/Uconn Each Semester/2019 Fall/Math 5671/Week9/data/rental.csv')
rental_df = rental_df.alias('rental_df')
rental_df.show(3)

+---------+-------------------+------------+-----------+-------------------+--------+-------------------+
|rental_id|        rental_date|inventory_id|customer_id|        return_date|staff_id|        last_update|
+---------+-------------------+------------+-----------+-------------------+--------+-------------------+
|        1|2005-05-24 22:53:30|         367|        130|2005-05-26 22:04:30|       1|2006-02-15 21:30:53|
|        2|2005-05-24 22:54:33|        1525|        459|2005-05-28 19:40:33|       1|2006-02-15 21:30:53|
|        3|2005-05-24 23:03:39|        1711|        408|2005-06-01 22:12:39|       1|2006-02-15 21:30:53|
+---------+-------------------+------------+-----------+-------------------+--------+-------------------+
only showing top 3 rows



In [12]:
staff_df = spark.read.format("csv").option("header", "true").load(r'/content/gdrive/My Drive/Uconn Each Semester/2019 Fall/Math 5671/Week9/data/staff.csv')
staff_df = staff_df.alias('staff_df')
staff_df.show(3)

+--------+----------+---------+----------+-------+--------------------+--------+------+--------+--------------------+--------------+
|staff_id|first_name|last_name|address_id|picture|               email|store_id|active|username|            password|   last_update|
+--------+----------+---------+----------+-------+--------------------+--------+------+--------+--------------------+--------------+
|       1|      Mike|  Hillyer|         3|   NULL|Mike.Hillyer@saki...|       1|     1|    Mike|8cb2237d0679ca88d...|2/15/2006 3:57|
|       2|       Jon| Stephens|         4|   NULL|Jon.Stephens@saki...|       2|     1|     Jon|                NULL|2/15/2006 3:57|
+--------+----------+---------+----------+-------+--------------------+--------+------+--------+--------------------+--------------+



In [13]:
inventory_df = spark.read.format("csv").option("header", "true").load(r'/content/gdrive/My Drive/Uconn Each Semester/2019 Fall/Math 5671/Week9/data/inventory.csv')
inventory_df = inventory_df.alias('inventory_df')
inventory_df.show(3)

+------------+-------+--------+-------------------+
|inventory_id|film_id|store_id|        last_update|
+------------+-------+--------+-------------------+
|           1|      1|       1|2006-02-15 05:09:17|
|           2|      1|       1|2006-02-15 05:09:17|
|           3|      1|       1|2006-02-15 05:09:17|
+------------+-------+--------+-------------------+
only showing top 3 rows



In [14]:
store_df = spark.read.format("csv").option("header", "true").load(r'/content/gdrive/My Drive/Uconn Each Semester/2019 Fall/Math 5671/Week9/data/store.csv')
store_df = store_df.alias('store_df')
store_df.show(3)

+--------+----------------+----------+-------------------+
|store_id|manager_staff_id|address_id|        last_update|
+--------+----------------+----------+-------------------+
|       1|               1|         1|2006-02-15 04:57:12|
|       2|               2|         2|2006-02-15 04:57:12|
+--------+----------------+----------+-------------------+



In [15]:
address_df = spark.read.format("csv").option("header", "true").load(r'/content/gdrive/My Drive/Uconn Each Semester/2019 Fall/Math 5671/Week9/data/address.csv')
address_df = address_df.alias('address_df')
address_df.show(3)

+----------+------------------+--------+--------+-------+-----------+-----------+-------------------+
|address_id|           address|address2|district|city_id|postal_code|      phone|        last_update|
+----------+------------------+--------+--------+-------+-----------+-----------+-------------------+
|         1| 47 MySakila Drive|    NULL| Alberta|    300|       null|       null|2014-09-25 22:30:27|
|         2|28 MySQL Boulevard|    NULL|     QLD|    576|       null|       null|2014-09-25 22:30:09|
|         3| 23 Workhaven Lane|    NULL| Alberta|    300|       null|14033335568|2014-09-25 22:30:27|
+----------+------------------+--------+--------+-------+-----------+-----------+-------------------+
only showing top 3 rows



In [16]:
customer_df = spark.read.format("csv").option("header", "true").load(r'/content/gdrive/My Drive/Uconn Each Semester/2019 Fall/Math 5671/Week9/data/customer.csv')
customer_df = customer_df.alias('customer_df')
customer_df.show(3)

+-----------+--------+----------+---------+--------------------+----------+------+-------------------+-------------------+
|customer_id|store_id|first_name|last_name|               email|address_id|active|        create_date|        last_update|
+-----------+--------+----------+---------+--------------------+----------+------+-------------------+-------------------+
|          1|       1|      MARY|    SMITH|MARY.SMITH@sakila...|         5|     1|2006-02-14 22:04:36|2006-02-15 04:57:20|
|          2|       1|  PATRICIA|  JOHNSON|PATRICIA.JOHNSON@...|         6|     1|2006-02-14 22:04:36|2006-02-15 04:57:20|
|          3|       1|     LINDA| WILLIAMS|LINDA.WILLIAMS@sa...|         7|     1|2006-02-14 22:04:36|2006-02-15 04:57:20|
+-----------+--------+----------+---------+--------------------+----------+------+-------------------+-------------------+
only showing top 3 rows



In [17]:
film_df = spark.read.format("csv").option("header", "true").load(r'/content/gdrive/My Drive/Uconn Each Semester/2019 Fall/Math 5671/Week9/data/film.csv')
film_df = film_df.alias('film_df')
film_df.show(3)

+-------+----------------+--------------------+------------+-----------+--------------------+---------------+-----------+------+----------------+------+--------------------+-------------------+
|film_id|           title|         description|release_year|language_id|original_language_id|rental_duration|rental_rate|length|replacement_cost|rating|    special_features|        last_update|
+-------+----------------+--------------------+------------+-----------+--------------------+---------------+-----------+------+----------------+------+--------------------+-------------------+
|      1|ACADEMY DINOSAUR|A Epic Drama of a...|        2006|          1|                NULL|              6|       0.99|    86|           20.99|    PG|Deleted Scenes,Be...|2006-02-15 05:03:42|
|      2|  ACE GOLDFINGER|A Astounding Epis...|        2006|          1|                NULL|              3|       4.99|    48|           12.99|     G|Trailers,Deleted ...|2006-02-15 05:03:42|
|      3|ADAPTATION HOLES|A As

In [0]:
# spark.stop()

# Question 1: 


In [70]:
payment_df.groupBy('customer_id').agg({'amount':'sum'}).show(3)

+-----------+------------------+
|customer_id|       sum(amount)|
+-----------+------------------+
|        296|129.69999999999993|
|        467|139.70999999999998|
|        125|135.67999999999998|
+-----------+------------------+
only showing top 3 rows



# Question 2: 




In [71]:
from pyspark.sql import functions as f

rental_df.filter(f.col("return_date") < f.unix_timestamp(f.lit('2015-01-01 00:00:00')).cast('timestamp')).orderBy('return_date', ascending=True).show(3)

+---------+-------------------+------------+-----------+-------------------+--------+-------------------+
|rental_id|        rental_date|inventory_id|customer_id|        return_date|staff_id|        last_update|
+---------+-------------------+------------+-----------+-------------------+--------+-------------------+
|       32|2005-05-25 04:06:21|        3832|        230|2005-05-25 23:55:21|       1|2006-02-15 21:30:53|
|       21|2005-05-25 01:59:46|         146|        388|2005-05-26 01:01:46|       2|2006-02-15 21:30:53|
|       14|2005-05-25 00:31:15|        2701|        446|2005-05-26 02:56:15|       1|2006-02-15 21:30:53|
+---------+-------------------+------------+-----------+-------------------+--------+-------------------+
only showing top 3 rows



# Question 3



In [72]:
rental_df.groupBy('staff_id').agg({'rental_id':'count'}).show()

+--------+----------------+
|staff_id|count(rental_id)|
+--------+----------------+
|       1|            8040|
|       2|            8004|
+--------+----------------+



# Question 4

In [73]:
from pyspark.sql.functions import concat, col, lit

# creat new dataframe 
  
new = staff_df.select(concat(staff_df['staff_id']).alias('staff_id'), concat(staff_df['first_name']).alias('first_name'), concat(staff_df['last_name']).alias('last_name'))
new.show()



# dataframe in question 3
new2 = rental_df.groupBy('staff_id').agg({'rental_id':'count'})
new2.show()

# answer:  join two dataframe according to staff_id
new.join(new2,"staff_id").show()




+--------+----------+---------+
|staff_id|first_name|last_name|
+--------+----------+---------+
|       1|      Mike|  Hillyer|
|       2|       Jon| Stephens|
+--------+----------+---------+

+--------+----------------+
|staff_id|count(rental_id)|
+--------+----------------+
|       1|            8040|
|       2|            8004|
+--------+----------------+

+--------+----------+---------+----------------+
|staff_id|first_name|last_name|count(rental_id)|
+--------+----------+---------+----------------+
|       1|      Mike|  Hillyer|            8040|
|       2|       Jon| Stephens|            8004|
+--------+----------+---------+----------------+



# Question 5

In [91]:
#1. use the rental_df as base to join inventory_df using "inventory_id"
q5df1 = rental_df.join(inventory_df,"inventory_id") 

#2. join with store_df using "store_id"
q5df2 = q5df1.join(store_df,"store_id") 
 
#3. join with address_df using "address_id"
q5df3 = q5df2.join(address_df,"address_id") 

#4. join with customer first and last name 
q5df4 = q5df3.join(new,"staff_id") 

#5. join with film data to get film name and description using "film_df"
q5df5 = q5df4.join(film_df,"film_id")

#6. order by return_date in ascending order 
q5final = q5df5.orderBy('return_date', ascending=True)
q5final.show(3)

+-------+--------+----------+--------+------------+---------+-------------------+-----------+-------------------+-------------------+-------------------+----------------+-------------------+------------------+--------+--------+-------+-----------+-----+-------------------+----------+---------+------------------+--------------------+------------+-----------+--------------------+---------------+-----------+------+----------------+------+--------------------+-------------------+
|film_id|staff_id|address_id|store_id|inventory_id|rental_id|        rental_date|customer_id|        return_date|        last_update|        last_update|manager_staff_id|        last_update|           address|address2|district|city_id|postal_code|phone|        last_update|first_name|last_name|             title|         description|release_year|language_id|original_language_id|rental_duration|rental_rate|length|replacement_cost|rating|    special_features|        last_update|
+-------+--------+----------+--------+

# Question *6*

In [108]:
# using the payment data
 
q6d1 = payment_df.groupBy('customer_id').agg({'amount':'sum'}) 
q6d1.filter(f.col("sum(amount)") > 200).show()


+-----------+-----------------+
|customer_id|      sum(amount)|
+-----------+-----------------+
|        526|221.5500000000001|
|        148|216.5400000000001|
+-----------+-----------------+



# Question 7

In [123]:
# From the customer dataframe get three related columns
q7d1 = customer_df.select(concat(customer_df['customer_id']).alias('customer_id'), concat(customer_df['first_name']).alias('first_name'), concat(customer_df['last_name']).alias('last_name'))
 
# join the results with q6d1
q7d2 = q7d1.join(q6d1,"customer_id") 
q7d3 = q7d2.filter(f.col("sum(amount)") > 200) 
q7d3.show()
# get rid of first colmn and get final answer
q7d4 = q7d3.drop("customer_id")
q7d4.show()




+-----------+----------+---------+-----------------+
|customer_id|first_name|last_name|      sum(amount)|
+-----------+----------+---------+-----------------+
|        148|   ELEANOR|     HUNT|216.5400000000001|
|        526|      KARL|     SEAL|221.5500000000001|
+-----------+----------+---------+-----------------+

+----------+---------+-----------------+
|first_name|last_name|      sum(amount)|
+----------+---------+-----------------+
|   ELEANOR|     HUNT|216.5400000000001|
|      KARL|     SEAL|221.5500000000001|
+----------+---------+-----------------+



# Question 8


In [155]:
# Create dataset that contains a list of customer_id that rented in 2005
q8_1= rental_df.filter(f.col("rental_date") < f.unix_timestamp(f.lit('2006-01-01 00:00:00')).cast('timestamp')).orderBy('return_date', ascending=True) 
q8_2005= q8_1.filter(f.col("return_date") > f.unix_timestamp(f.lit('2005-01-01 00:00:00')).cast('timestamp')).orderBy('return_date', ascending=True)
 
q8_2005.show(3)

# Create dataset that contains a list of customer_id that rented in 2006
q8_3= rental_df.filter(f.col("rental_date") < f.unix_timestamp(f.lit('2007-01-01 00:00:00')).cast('timestamp')).orderBy('return_date', ascending=True) 
q8_2006= q8_3.filter(f.col("return_date") > f.unix_timestamp(f.lit('2006-01-01 00:00:00')).cast('timestamp')).orderBy('return_date', ascending=True)
q8_2006.show(3)
 
# (a): create a DataFrame of customer_id that exist in the 2005 DataFrame but not in the 2006 DataFrame
# merge using the payment_id from payment_df
q8_2005_first = q8_2005.join(payment_df,"customer_id") 
question8_a= q8_2005_first.filter(f.col("return_date") < f.unix_timestamp(f.lit('2006-01-01 00:00:00')).cast('timestamp')).orderBy('return_date', ascending=True) 
question8_a2= question8_a.filter(f.col("return_date") > f.unix_timestamp(f.lit('2005-01-01 00:00:00')).cast('timestamp')).orderBy('return_date', ascending=True)
question8_a2.show()

 
 
# (a): create a DataFrame of customer_id that exist in the 2005 DataFrame but not in the 2006 DataFrame
# merge using the payment_id from payment_df
q8_2006_first = q8_2006.join(payment_df,"customer_id") 
question8_b= q8_2006_first.filter(f.col("return_date") < f.unix_timestamp(f.lit('2007-01-01 00:00:00')).cast('timestamp')).orderBy('return_date', ascending=True) 
question8_b2= question8_b.filter(f.col("return_date") > f.unix_timestamp(f.lit('2006-01-01 00:00:00')).cast('timestamp')).orderBy('return_date', ascending=True)
question8_b2.show()


+---------+-------------------+------------+-----------+-------------------+--------+-------------------+
|rental_id|        rental_date|inventory_id|customer_id|        return_date|staff_id|        last_update|
+---------+-------------------+------------+-----------+-------------------+--------+-------------------+
|       32|2005-05-25 04:06:21|        3832|        230|2005-05-25 23:55:21|       1|2006-02-15 21:30:53|
|       21|2005-05-25 01:59:46|         146|        388|2005-05-26 01:01:46|       2|2006-02-15 21:30:53|
|       14|2005-05-25 00:31:15|        2701|        446|2005-05-26 02:56:15|       1|2006-02-15 21:30:53|
+---------+-------------------+------------+-----------+-------------------+--------+-------------------+
only showing top 3 rows

+---------+-------------------+------------+-----------+-----------+--------+-------------------+
|rental_id|        rental_date|inventory_id|customer_id|return_date|staff_id|        last_update|
+---------+-------------------+------